In [1]:
%cd drive/My\ Drive/Colab\ Notebooks/DLG/semantic-mediawiki-page-creation/semantic-mediawiki-page-creation/sql-notebooks

/content/drive/My Drive/Colab Notebooks/DLG/semantic-mediawiki-page-creation/semantic-mediawiki-page-creation/sql-notebooks


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from getpass import getpass
pjt_code = getpass('ProjectCode: ')

ProjectCode: ··········


In [17]:
%%bigquery --project {pjt_code} df_user_master
DECLARE start_date DATE DEFAULT "2019-01-01";
DECLARE end_date DATE DEFAULT "2020-12-31";

WITH
user_master_table AS (
  SELECT
    user_id, real_name_normalized AS user_name, target_date
  FROM
    mgm.user
  WHERE
      target_date >= start_date AND target_date <= end_date AND
      display_name_normalized IS NULL -- use real name when display name is null
  UNION ALL
  SELECT
    user_id, display_name_normalized AS user_name, target_date
  FROM
    mgm.user
  WHERE
      target_date >= start_date AND target_date <= end_date AND
      display_name_normalized IS NOT NULL -- use display name when not null
  ORDER BY target_date, user_id
),
user_last_date AS (
  SELECT
    user_id,
    MAX(target_date) = (SELECT MAX(target_date) FROM user_master_table) AS is_in_last_date
  FROM
    user_master_table
  GROUP BY
    user_id
)

SELECT
  m.user_id, m.user_name, m.target_date, l.is_in_last_date
FROM
  user_master_table m
  LEFT JOIN
    user_last_date l
  ON m.user_id = l.user_id
;

In [18]:
df_user_master.to_csv('user_master.csv', index = False)

,user_id,user_name,target_date,is_in_last_date
0,UL70KNF4P,S,2019-09-09,False
1,UMFDK5ETW,Y,2019-09-09,False
2,UMC518ER0,Yan,2019-09-09,True
3,UMG3YGKJQ,ito,2019-09-09,False
4,UNHCE37QU,kjm,2019-09-09,False
...,...,...,...,...
106019,UQ5P3BKJN,Hiroki,2019-07-15,False
106020,UQGMHCCJ2,尾銭泰徳 ozeni.yasunori,2019-07-15,True
106021,UQJ5KR30A,Kubo Ontheair,2019-07-15,True
106022,UQM7K28R3,rakutarou,2019-07-15,False
